In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc,get_scorer_names
from sklearn.metrics import make_scorer, matthews_corrcoef,accuracy_score, classification_report,confusion_matrix,ConfusionMatrixDisplay, roc_curve, matthews_corrcoef, balanced_accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from imblearn.pipeline import Pipeline as IMBPipeline
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import numpy as np
import random
import os

In [2]:
# Función para definir en un solo lugar todas las semillas
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

seed_value = 42
seed_everything(seed_value)

In [3]:
#Carga el archivo en un dataframe
file_path = 'postprandial_hypoglycemias.csv'
df = pd.read_csv(file_path)

In [4]:
#Seleccionamos las features y eliminamos los resgistros con nulos
physiological_features = ['glucose','cho','next_iob','cob','aob_3h','aob_6h']
physiological_features_no_glucose = ['cho','iob','next_iob','cob','aob_3h','aob_6h']
no_glucose_related_features = ['glucose','cho', 'basal', 'bolus_normal', 'total_bolus_squared','cob','next_iob','aob_3h','aob_6h']
no_glucose_related_features_exclude_aob = ['glucose','cho', 'basal', 'bolus_normal', 'total_bolus_squared','cob','next_iob']
more_features = ['glucose','glucose_range','glucose_rate_change','mean_glucose_1h','glucose_1h','glucose_30m','cho', 'basal', 'bolus_normal', 'total_bolus_squared','cob','next_iob','aob_3h','aob_6h']
more_features_exclude_aob = ['glucose','glucose_range','glucose_rate_change','mean_glucose_1h','glucose_1h','glucose_30m','cho', 'basal', 'bolus_normal', 'total_bolus_squared','cob','next_iob']
more_features_no_cob_no_iob = ['glucose','glucose_range','glucose_rate_change','mean_glucose_1h','glucose_1h','glucose_30m','cho', 'basal', 'bolus_normal', 'total_bolus_squared']
all_features = ['glucose','glucose_range','glucose_rate_change','mean_glucose_1h','glucose_1h','glucose_30m','cho', 'basal', 'bolus_normal', 'total_bolus_squared',
                'cob','next_iob','iob','aob_3h','aob_6h','hour']
selected_features = all_features
target = 'hypoglycemia_severe'
df = df[selected_features + [target]]
df = df.dropna()

In [5]:
# Separamos las características (X) i la etiqueta (y)
X = df.drop(target, axis=1)
y = df[target]

In [6]:
print('Registros totales para entrenamiento: ' + str(len(df)))
print('Registros totales para entrenamiento con hipoglucemia severa: ' + str(len(df[df[target] == 1])))


Registros totales para entrenamiento: 654
Registros totales para entrenamiento con hipoglucemia severa: 27


**Optimización de parámetros mediante GridSearchCV**

In [7]:
# Definir la función de especificidad
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return specificity

# Crear un pipeline con SMOTE y SVC
model_pipeline = IMBPipeline([
    ('smote', SMOTE(random_state=seed_value)),
    ('model', SVC(kernel='rbf', random_state=seed_value))
])

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'smote__k_neighbors': [3, 5, 7],
    'model__C': [0.1, 1, 10, 100],
    'model__gamma': [0.001, 0.01, 0.1, 1]
}

# Definir los scorers
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'mcc': make_scorer(matthews_corrcoef),
    'specificity': make_scorer(specificity_score)
}

# Configurar GridSearchCV
grid_search = GridSearchCV(model_pipeline, param_grid, cv=StratifiedKFold(n_splits=5), scoring=scoring, refit='f1', verbose=3)

# Realizar la búsqueda de hiperparámetros
grid_search.fit(X, y)

# Mostrar los mejores parámetros y el mejor score
print("Mejores parámetros: ", grid_search.best_params_)
print("Mejor score (F1): ", grid_search.best_score_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END model__C=0.1, model__gamma=0.001, smote__k_neighbors=3; accuracy: (test=0.962) f1: (test=0.000) mcc: (test=0.000) precision: (test=0.000) recall: (test=0.000) roc_auc: (test=0.632) specificity: (test=1.000) total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV 2/5] END model__C=0.1, model__gamma=0.001, smote__k_neighbors=3; accuracy: (test=0.870) f1: (test=0.105) mcc: (test=0.060) precision: (test=0.071) recall: (test=0.200) roc_auc: (test=0.635) specificity: (test=0.897) total time=   0.1s
[CV 3/5] END model__C=0.1, model__gamma=0.001, smote__k_neighbors=3; accuracy: (test=0.947) f1: (test=0.222) mcc: (test=0.211) precision: (test=0.333) recall: (test=0.167) roc_auc: (test=0.603) specificity: (test=0.984) total time=   0.1s
[CV 4/5] END model__C=0.1, model__gamma=0.001, smote__k_neighbors=3; accuracy: (test=0.947) f1: (test=0.000) mcc: (test=-0.019) precision: (test=0.000) recall: (test=0.000) roc_auc: (test=0.619) specificity: (test=0.992) total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV 5/5] END model__C=0.1, model__gamma=0.001, smote__k_neighbors=3; accuracy: (test=0.962) f1: (test=0.000) mcc: (test=0.000) precision: (test=0.000) recall: (test=0.000) roc_auc: (test=0.755) specificity: (test=1.000) total time=   0.1s
[CV 1/5] END model__C=0.1, model__gamma=0.001, smote__k_neighbors=5; accuracy: (test=0.962) f1: (test=0.000) mcc: (test=0.000) precision: (test=0.000) recall: (test=0.000) roc_auc: (test=0.640) specificity: (test=1.000) total time=   0.1s
[CV 2/5] END model__C=0.1, model__gamma=0.001, smote__k_neighbors=5; accuracy: (test=0.840) f1: (test=0.087) mcc: (test=0.036) precision: (test=0.056) recall: (test=0.200) roc_auc: (test=0.610) specificity: (test=0.865) total time=   0.1s
[CV 3/5] END model__C=0.1, model__gamma=0.001, smote__k_neighbors=5; accuracy: (test=0.947) f1: (test=0.222) mcc: (test=0.211) precision: (test=0.333) recall: (test=0.167) roc_auc: (test=0.651) specificity: (test=0.984) total time=   0.1s
[CV 4/5] END model__C=0.1, model__gamma=0.00

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV 2/5] END model__C=100, model__gamma=1, smote__k_neighbors=7; accuracy: (test=0.962) f1: (test=0.000) mcc: (test=0.000) precision: (test=0.000) recall: (test=0.000) roc_auc: (test=0.500) specificity: (test=1.000) total time=   0.1s
[CV 3/5] END model__C=100, model__gamma=1, smote__k_neighbors=7; accuracy: (test=0.954) f1: (test=0.000) mcc: (test=0.000) precision: (test=0.000) recall: (test=0.000) roc_auc: (test=0.500) specificity: (test=1.000) total time=   0.1s


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV 4/5] END model__C=100, model__gamma=1, smote__k_neighbors=7; accuracy: (test=0.954) f1: (test=0.000) mcc: (test=0.000) precision: (test=0.000) recall: (test=0.000) roc_auc: (test=0.500) specificity: (test=1.000) total time=   0.1s
[CV 5/5] END model__C=100, model__gamma=1, smote__k_neighbors=7; accuracy: (test=0.962) f1: (test=0.000) mcc: (test=0.000) precision: (test=0.000) recall: (test=0.000) roc_auc: (test=0.500) specificity: (test=1.000) total time=   0.1s
Mejores parámetros:  {'model__C': 1, 'model__gamma': 0.001, 'smote__k_neighbors': 5}
Mejor score (F1):  0.11587301587301586


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Media métricas validación cruzada**

In [8]:
# Crear un pipeline que primero aplica SMOTE y luego el modelo
model_pipeline = IMBPipeline([
    ('smote', SMOTE(random_state=seed_value, k_neighbors=5)),
    ('model', make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1, gamma=0.001)))
])

# Usar StratifiedKFold para mantener la proporción de clases en cada fold
cv = StratifiedKFold(n_splits=5)
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return specificity

# Crear un scorer personalizado para la especificidad
specificity_scorer = make_scorer(specificity_score)

scoring = {'accuracy' : 'accuracy', 'sensibility' : 'recall','specificity': specificity_scorer, 'roc_auc':'roc_auc','f1':'f1', 'mcc': make_scorer(matthews_corrcoef) }

# Realizar la validación cruzada
results = cross_validate(model_pipeline, X, y, cv=cv, scoring=scoring)

mean_scores = {metric: results['test_' + metric].mean().round(2) for metric in scoring}
std_scores = {metric: results['test_' + metric].std().round(2) for metric in scoring}

results_df = pd.DataFrame({
    'Metric': mean_scores.keys(),
    'Mean': mean_scores.values(),
    'Standard Deviation': std_scores.values()
})

print(results_df)

        Metric  Mean  Standard Deviation
0     accuracy  0.58                0.25
1  sensibility  0.61                0.37
2  specificity  0.58                0.27
3      roc_auc  0.71                0.11
4           f1  0.15                0.10
5          mcc  0.15                0.17
